# Generalized Forum Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import nltk
import gensim

Be careful to run the cells in order

Function that finds the tags allowing to retrieve the data we need

In [382]:
def find_tag(x, comp, tags):
    tag = x.name
    try:
        class_name = x['class'][0]
        tags.append(class_name)
        tags.append(tag)
        next_ = x.findNext(tag, {'class': class_name})
    except:
        #print("TAG: ", tag)
        tags.append(tag)
        #print("TAGS: ", tags)
        next_ = x.findNext(tag)
        #print("next: ", next_)
        
    if comp in next_.text:
        print("I've found the right tag, it's ", tags[::-1])
        return tags[::-1]
    else:
        return find_tag(x.parent, comp, tags)

In [204]:
def find_next_tag():
    if soup.find(text = "Next") != None:
        next_tag = soup.find(text = "Next").parent.name
        print("The tag for next page is ", next_tag)
        return next_tag
    elif soup.find(text = "next") != None:
        next_tag = soup.find(text = "next").parent.name
        print("The tag for next page is ", next_tag)
        return next_tag
    else:
        print("tag of the next page is not treated in this program")
        return None

In [421]:
def find_next(soup, next_tag):
    try:
        #check if find instead of find_all always work if not do [0]['href'][1:]
        if next_tag is not None:
            return soup.find_all(next_tag, string="Next")[0]['href'][1:]       #also work with "next"
        return ""
    except:
        return ""

def collect_forum_data(soup_row, tags, soup_row_ok = 0, link=""):
    
    data = {}
    try:
        if not soup_row_ok:         #change with length
            soup_row = soup_row.find(tags[len(tags) -1])
    
        data['Title'] = soup_row.text.strip()
        data['Link'] = soup_row['href'][1:]
        
    except:
        pass
    return data

def collect_post_data(soup_row, tags, soup_row_ok, link=""):
    data = {}
    try:          #check case tags==3, 2
        if len(tags) == 2 or len(tags) == 3:            #case where len == 2 and no class will fail (should do a second fin)            
            soup_row = soup_row.find(tags[len(tags)-1])
        elif len(tags) == 4:
            soup_row = soup_row.find(tags[2]).find(tags[3])    
        
        data['Link'] = link.replace(PREFIX_URL, '')

        try:
            data['Username'] = soup_row.text.strip()
        except:
            pass
    except:
        pass
    return data

def collect_recursively(data, soup, tags, next_tag, fcte_name, link="", index=""):
    try:
        if index:
            print(index, end='\r', flush=True)
        
        soup_row_ok = 0
        if len(tags) == 2:
            soup_rows = soup.find_all(tags[0], {'class': tags[1]})
            if not soup_rows:
                soup_rows = soup.find_all(tags[0]) 
            else:
                soup_row_ok = 1      
        if len(tags) == 3 or len(tags) == 4:
            soup_rows = soup.find_all(tags[0], {'class': tags[1]})
    
        
        #print(soup_rows)
        data.extend([fcte_name(soup_row, tags, soup_row_ok, link) for soup_row in soup_rows])
        next_url = find_next(soup, next_tag)           #put a condition (something with next_tag == None)
        if next_url:
            soup = BeautifulSoup(requests.get(PREFIX_URL + next_url).text, 'html.parser')
            if index:
                return collect_recursively(data, soup, tags, next_tag, fcte_name, link, index+1)
            else:
                return collect_recursively(data, soup, tags, next_tag, fcte_name, link)
        else:
            return data
    except:
        pass
    return data
                
def verify_if_treated(soup, tags):
    if len(tags) > 4 or len(tags) < 2:
        print("This case is not treated yet")
        
    if len(tags) == 3 or len(tags) == 4:
        try:
            soup.find(tags[0], {'class': tags[1]})
        except:
            print("This case is not treated yet")
    
def collect_all_links(soup, tags, next_tag, fcte_name):      #soup or url? Combine (put verifications also to collect)
    verify_if_treated(soup, tags)

    data = collect_recursively([], soup, tags, next_tag, fcte_name)      #get rid of 1?
    return pd.DataFrame(data).dropna()

def collect(forum_df, tags, fcte_name):
    data = []
    total = len(forum_df['Link'])
    index = 0
    for url in forum_df['Link']:
        index += 1
        print('{} out of {}'.format(index, total), end='\r', flush=True)
        #url = forum_df['Link'][0]
        soup = BeautifulSoup(requests.get(PREFIX_URL + url).text, 'html.parser')
        verify_if_treated(soup, tags)
        data.extend(collect_recursively([], soup, tags, next_tag, fcte_name, url))
    return pd.DataFrame(data)

1. Collect all titles and links of the forum

Enter the 3 parameters needed:

In [413]:
#Holiday Truths
#PREFIX_URL = 'https://www.holidaytruths.co.uk/'
#START_URL = PREFIX_URL + 'forum/america-canada-discussion-forum-f2-0.html'
#soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#title1 = 'ESTA question on employment'
#title2 = 'Vegas Buffets/Restaurants'

#Wrong Planet
#PREFIX_URL = 'http://wrongplanet.net/forums'
#START_URL = PREFIX_URL + '/viewforum.php?f=19'
#soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#title1 = 'RE: Kids w/ Classic Autism, PDD-NOS & Speech Delays'
#title2 = 'Parents on the spectrum'

#Stack Overflow
#PREFIX_URL = 'https://stackoverflow.com/'
#START_URL = PREFIX_URL + 'questions/tagged/forum'
#soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#title1 = 'Should DynamoDB adjacency lists use discrete partition keys to model each type of relationship?'
#title2 = 'How to Bypass [hide] element in Forums?'

#Au Féminin
#PREFIX_URL = 'https://astrologie.aufeminin.com/forum'
#START_URL = PREFIX_URL + '/all'
#soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#title1 = 'coucou....'
#title2 = 'échange serieux en mp'


#Trip Advisor
PREFIX_URL = 'https://www.tripadvisor.co.uk/'
START_URL = PREFIX_URL + 'ShowForum-g1-i12334-Holiday_Travel.html'
soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
title1 = 'See TOP QUESTIONS before posting!'
title2 = 'Use the SEARCH BOX function before posting!'

Find the tags

In [414]:
import re
try:
    right_tags = find_tag(soup.find(text=re.compile(title1)).parent, title2, [])
    next_tag = find_next_tag()
except:
    print("Sorry I couldn't find the tags")

I've found the right tag, it's  ['b', 'a']
tag of the next page is not treated in this program


In [423]:
threads = collect_all_links(soup, right_tags, next_tag, collect_forum_data)
#print(threads)

2. Collect all usernames, messages in every link

Enter the parameters needed:

In [424]:
START_URL = PREFIX_URL + threads.Link[0]
soup =  BeautifulSoup(requests.get(START_URL).text, 'html.parser')

#Holiday Truths
#user1 = 'AnnaM'
#user2 = 'Glynis HT Admin'

#Wrong Planet
#user1 = 'cyberdad'
#user2 = 'Solvejg'

#Trip Advisor
#print(soup.prettify())
user1 = 'BradJill'
user2 = 'Eden7'

In [425]:
right_tags = find_tag(soup.find(text = user1).parent, user2, [])
next_tag = find_next_tag()

I've found the right tag, it's  ['div', 'username', 'a', 'span']
tag of the next page is not treated in this program


In [426]:
threads = threads[:21]
posts = collect(threads, right_tags, collect_post_data)

In [427]:
print(posts)

                                                  Link              Username
0    ShowTopic-g1-i12334-k7867029-See_TOP_QUESTIONS...              BradJill
1    ShowTopic-g1-i12334-k7867029-See_TOP_QUESTIONS...                 Eden7
2    ShowTopic-g1-i12334-k7867029-See_TOP_QUESTIONS...              BradJill
3    ShowTopic-g1-i12334-k7867031-Use_the_SEARCH_BO...              BradJill
4    ShowTopic-g1-i12334-k7867031-Use_the_SEARCH_BO...               KVE1005
5    ShowTopic-g1-i12334-k7867031-Use_the_SEARCH_BO...                 Eden7
6    ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...              BradJill
7    ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...               Super M
8    ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...          thegoodwitch
9    ShowTopic-g1-i12334-k5876122-How_to_Use_the_Ho...                 Eden7
10   ShowTopic-g1-i12334-k11057297-Tui_Uk_website-H...                luxsey
11   ShowTopic-g1-i12334-k11057297-Tui_Uk_website-H...             LeyLand10